In [1]:
from bs4 import BeautifulSoup
import urllib
from datetime import datetime
from datetime import timedelta
import pandas as pd
import pickle
import numpy as np
import pandas as pd
import nltk
from sklearn.externals import joblib
import re
import codecs
from sklearn import feature_extraction
import os
from bs4 import BeautifulSoup
import urllib
from nltk.stem.snowball import SnowballStemmer
import scipy.spatial.distance
import numba 
from numba import jit



nyt_unique=pd.read_csv('nyt_uniques_topicsLabeled.csv')
reuters_unique = pickle.load( open( "20180516-20180621_reuters_unique.pkl", "rb" ) )

def token_stem(text):
    # tokenize by sentence and word. this way you ensure you get rid of punctuations
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    tokens_with_letters = []
    # use the regex library to search only for items that contain letters. this will enable you to eliminate punctuation
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            tokens_with_letters.append(token)
    stems = [stemmer.stem(t) for t in tokens_with_letters] #"stems" part
    return stems

reuters_unique.reset_index(inplace=True)
reuters_unique.drop('index',axis=1,inplace=True)
nyt_labeled_only=nyt_unique.loc[nyt_unique['Section'] != 'Other'].copy()
nyt_labeled_only.reset_index(inplace=True)
nyt_labeled_only.drop('index',axis=1,inplace=True)


nyt_articles=nyt_labeled_only['Article'].tolist()
reuters_articles=reuters_unique['article'].tolist()
nyt_articles.extend(reuters_articles)

nyt_articles=list(map(lambda unicode_line:unicode_line.translate({ord(c): None for c in 'Äòôúîù'}),nyt_articles))


stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("english") 
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english',use_idf=True, norm='l2',tokenizer=token_stem, ngram_range=(1,3))
tfidf_matrix = vectorizer.fit_transform(nyt_articles)
tfidf_1=tfidf_matrix[0:1400,:]
tfidf_2=tfidf_matrix[1400:,:]
print(type(tfidf_matrix))


from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

#%time similarities_sparse = cosine_similarity(tfidf_1,tfidf_2,dense_output=False)
%time similarities_dense = cosine_similarity(tfidf_1,tfidf_2,dense_output=True)
np.count_nonzero(similarities_dense==0)

<class 'scipy.sparse.csr.csr_matrix'>
CPU times: user 5.18 s, sys: 416 ms, total: 5.59 s
Wall time: 2.61 s


347046

In [2]:
similarities_dense.shape

(1400, 22322)

In [3]:
import pickle 
import numpy as np
import pandas as pd
article_similarities = pickle.load( open( "article_similarities.pkl", "rb" ) )

In [4]:
article_distances=1-article_similarities

In [5]:
big_df = pickle.load( open( "big_df.pkl", "rb" ) )
flatten = lambda l: [item for sublist in l for item in sublist]
big_df['article_distances']=pd.Series(flatten(article_distances.tolist()))
big_df.shape

(31250800, 17)

In [6]:
wf=open('big_df.pkl','wb')
pickle.dump(big_df,wf)